# GlassAlpha Quickstart: Your First Audit in 8 Minutes

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GlassAlpha/glassalpha/blob/main/examples/notebooks/quickstart_colab.ipynb)

This notebook demonstrates how to generate a professional ML audit report in under 8 minutes using GlassAlpha's `from_model()` API.

**What you'll learn:**
- Train a simple credit scoring model
- Generate a comprehensive audit with one function call
- View inline audit results
- Understand fairness and performance metrics

**Time**: ~8 minutes | **Difficulty**: Beginner


## Step 1: Install GlassAlpha (30 seconds)

Run this cell to install GlassAlpha and its dependencies.


In [ ]:
%pip install -q glassalpha[explain]


## Step 2: Environment Setup (15 seconds)

Import libraries and set random seeds for reproducibility.


In [ ]:
"""Environment verification for reproducibility"""
import sys, platform, random, numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import glassalpha as ga

# Set random seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

print({
    "python": sys.version.split()[0],
    "platform": platform.platform(),
    "glassalpha": getattr(ga, "__version__", "dev"),
    "seed": SEED
})


## Step 3: Load Data (1 minute)

We'll use the German Credit dataset - a classic benchmark for credit risk assessment.


In [ ]:
# Load German Credit dataset
from glassalpha.datasets import load_german_credit

# Load preprocessed data
data = load_german_credit()
df = data['dataframe']

print(f"Dataset shape: {df.shape}")
print(f"\nTarget distribution:\n{df['credit_risk'].value_counts(normalize=True)}")
print(f"\nProtected attributes: {data['protected_attributes']}")

# Show first few rows
df.head()


## Step 4: Train Model (2 minutes)

Train a simple Random Forest classifier for credit risk prediction.


In [ ]:
# Prepare features and target
target_col = 'credit_risk'
protected_attrs = ['gender', 'age_group', 'foreign_worker']

# Features (exclude target and protected attributes for training)
feature_cols = [col for col in df.columns if col != target_col and col not in protected_attrs]
X = df[feature_cols]
y = df[target_col]

# Train/test split (stratified by target and gender for fairness analysis)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=SEED,
    stratify=df[[target_col, 'gender']]
)

# Train Random Forest model
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=5,
    random_state=SEED,
    n_jobs=-1
)

print("Training model...")
model.fit(X_train, y_train)

# Quick performance check
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"✓ Training accuracy: {train_score:.3f}")
print(f"✓ Test accuracy: {test_score:.3f}")
print(f"✓ Model ready for audit!")


## Step 5: Generate Audit with `from_model()` (3 minutes)

This is where the magic happens! One function call generates a comprehensive audit.


In [ ]:
# Generate comprehensive audit
result = ga.audit.from_model(
    model=model,
    X_test=X_test,
    y_test=y_test,
    protected_attributes={
        'gender': df.loc[X_test.index, 'gender'],
        'age_group': df.loc[X_test.index, 'age_group'],
        'foreign_worker': df.loc[X_test.index, 'foreign_worker']
    },
    random_seed=SEED
)

print("✓ Audit completed successfully!")


## Step 6: View Inline Results (30 seconds)

Display the audit summary directly in the notebook.


In [ ]:
# Display inline HTML summary
result


## Step 7: Explore Metrics (1 minute)

Access specific metrics programmatically.


In [ ]:
# Performance metrics
print("📊 PERFORMANCE METRICS")
print(f"  Accuracy: {result.performance.accuracy:.3f}")
print(f"  Precision: {result.performance.precision:.3f}")
print(f"  Recall: {result.performance.recall:.3f}")
print(f"  F1 Score: {result.performance.f1:.3f}")
print(f"  AUC-ROC: {result.performance.auc_roc:.3f}")

# Fairness metrics
print("\n⚖️  FAIRNESS METRICS")
print(f"  Demographic parity (gender): {result.fairness.demographic_parity_difference:.3f}")
print(f"  Equal opportunity (gender): {result.fairness.equal_opportunity_difference:.3f}")

# Check for bias
if result.fairness.has_bias():
    print("  ⚠️  Bias detected! Review fairness analysis.")
else:
    print("  ✅ No significant bias detected.")

# Calibration
print("\n🎯 CALIBRATION")
print(f"  Expected Calibration Error: {result.calibration.expected_calibration_error:.3f}")
print(f"  Brier Score: {result.calibration.brier_score:.3f}")


## Step 8: Visualize Results (optional)

Generate publication-quality plots.


In [ ]:
# Plot calibration curve
result.calibration.plot()

# Plot fairness metrics
result.fairness.plot_group_metrics()

# Plot confusion matrix
result.performance.plot_confusion_matrix()


## Step 9: Export PDF Report (optional)

Generate a professional PDF for regulatory submission.


In [ ]:
# Export to PDF
result.to_pdf("german_credit_audit.pdf")

print("✓ PDF report generated: german_credit_audit.pdf")
print("\nDownload the PDF from the Colab files panel (left sidebar) →")


## 🎉 Congratulations!

You've just generated your first ML audit in under 8 minutes!

### What you learned:
- ✅ Train a credit scoring model
- ✅ Generate comprehensive audit with `from_model()`
- ✅ View inline results in notebooks
- ✅ Access metrics programmatically
- ✅ Export professional PDF reports

### Next steps:
1. **Try with your own data**: See [Custom Data Guide](https://glassalpha.com/getting-started/custom-data/)
2. **Explore advanced features**: [Dataset Bias Detection](https://glassalpha.com/guides/dataset-bias/), [Shift Testing](https://glassalpha.com/guides/shift-testing/)
3. **Integrate into production**: [ML Engineer Workflow](https://glassalpha.com/guides/ml-engineer-workflow/)
4. **Learn about compliance**: [SR 11-7 Mapping](https://glassalpha.com/compliance/sr-11-7-mapping/), [EU AI Act](https://glassalpha.com/compliance/eu-ai-act-mapping/)

### Resources:
- **Documentation**: [glassalpha.com/docs](https://glassalpha.com/docs)
- **GitHub**: [github.com/GlassAlpha/glassalpha](https://github.com/GlassAlpha/glassalpha)
- **Discussions**: [github.com/GlassAlpha/glassalpha/discussions](https://github.com/GlassAlpha/glassalpha/discussions)

### Questions?
- Check the [FAQ](https://glassalpha.com/reference/faq/)
- Join [GitHub Discussions](https://github.com/GlassAlpha/glassalpha/discussions)
- Email: [contact@glassalpha.com](mailto:contact@glassalpha.com)
